<a href="https://colab.research.google.com/github/Krish7Hack/Side-Channel-Attack-Resistant-Cryptographic-Libraries/blob/main/Side_Channel_Attack_Resistant_Cryptographic_Libraries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Side-Channel Attack Resistant Cryptographic Libraries
# =====================================================
# This notebook implements constant-time versions of AES, RSA, and ECC,
# and evaluates their resistance to timing side-channel attacks.


In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import secrets
import random
from concurrent.futures import ThreadPoolExecutor

# Helper functions for timing measurements

In [ ]:
def measure_execution_time(func,*args,**kwargs):
  "Measure execution time of a function in nanoseconds"
